<a href="https://colab.research.google.com/github/hawc2/contentdm-iiif-api/blob/main/IIIF_OPENCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IIIF OPEN CV

https://codingdavinci.de/events/ost/

https://github.com/leanderseige/cdvost2018_iiifworkshop

This notebook makes use of public IIIF data from the National Gallery of Art.

Adapt for TUL ContentDM IIIF

### Step 1: Import some libraries.

In [1]:
%matplotlib inline
import numpy as np
import urllib.request
from matplotlib import pyplot as plt
import cv2
import json
from IPython.display import Image, display
from IPython.core.display import HTML 
import ipywidgets as widgets
from random import randint

### Step 2: Read IIIF Image API endpoints from NGAs Highlights Manifest.

In [37]:
#url = "https://cdm16002.contentdm.oclc.org/iiif/info/p16002coll4/4/manifest.json"
url = "https://cdm16002.contentdm.oclc.org/iiif/info/p16002coll9/manifest.json"

In [10]:
resp = urllib.request.urlopen(url)
data = resp.read().decode("utf-8")
data = json.loads(data)
pid = 0
img_urls = {}

In [ ]:
for c in data['sequences'][0]['canvases']:
    img_urls[pid]=c['images'][0]['resource']['service']['@id']
    print (img_urls[pid])
    pid = pid +1
print ("Ready.")

### Step 3: Let's get us some generic progress bar.

In [12]:
progbar = widgets.IntProgress(
    value=0,
    min=0,
    max=pid,
    step=1,
    description='Progress:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal')

### Step 4: Define a function to read a binary image from the web.

In [13]:
def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

### Step 5: Read all images with 4% of their actual size.

In [16]:
display(progbar)

img_smalls = {}
for pid in img_urls:
    progbar.value = pid
    turl = img_urls[pid]+"/full/pct:15/0/default.jpg"
    img_smalls[pid] = url_to_image(turl)
    display(Image(url=turl, width=400, height=400))
    if pid == 1000:
        break

IntProgress(value=0, description='Progress:', max=4)

### Step 6: Get us a function to detect faces using the opencv lib.

In [23]:
def analyze_images(image):
    #faceCascade = cv2.CascadeClassifier("haarcascades/haarcascade_frontalface_alt.xml")
    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = faceCascade.detectMultiScale(
        image,
        scaleFactor=1.1,
        minNeighbors=3,
        minSize=(20, 20),
        flags = cv2.CASCADE_SCALE_IMAGE
    )
    return faces

### Step 7: Find all faces in the images.
Save all coodinates multiplied by 25 as we retrieved the images scaled to 4%.

In [24]:
display(progbar)

faces_xy = {}
fid = 0
for pid in img_smalls:
    progbar.value=pid
    faces = analyze_images(img_smalls[pid])
    for (x, y, w, h) in faces:
        faces_xy[fid] = {}
        faces_xy[fid]['pid'] = pid
        faces_xy[fid]['x'] = x*25
        faces_xy[fid]['y'] = y*25
        faces_xy[fid]['w'] = w*25
        faces_xy[fid]['h'] = h*25
        fid = fid +1
        
print ("Found {0} faces!".format(len(faces_xy)))

IntProgress(value=0, description='Progress:', max=4)

Found 9 faces!


### Step 8: Generate URLs to the facial regions for each recognized face.

In [25]:
for fid in faces_xy:
    pid = faces_xy[fid]['pid']
    x = faces_xy[fid]['x']
    y = faces_xy[fid]['y']
    w = faces_xy[fid]['w']
    h = faces_xy[fid]['h']
    url = img_urls[pid]+"/%d,%d,%d,%d/300,/0/native.jpg"%(x,y,w,h)
    faces_xy[fid]['quick_url'] = url

### Step 9: Display what we have done so far.

In [26]:
for fid in faces_xy:
    print(faces_xy[fid]['quick_url'])
    display(Image(url=faces_xy[fid]['quick_url']))
   

https://cdm16002.contentdm.oclc.org/digital/iiif/p16002coll4/1/4475,1800,1750,1750/300,/0/native.jpg


https://cdm16002.contentdm.oclc.org/digital/iiif/p16002coll4/1/10150,13750,1600,1600/300,/0/native.jpg


https://cdm16002.contentdm.oclc.org/digital/iiif/p16002coll4/2/12825,1000,1425,1425/300,/0/native.jpg


https://cdm16002.contentdm.oclc.org/digital/iiif/p16002coll4/2/10100,6000,1900,1900/300,/0/native.jpg


https://cdm16002.contentdm.oclc.org/digital/iiif/p16002coll4/2/12900,9925,1525,1525/300,/0/native.jpg


https://cdm16002.contentdm.oclc.org/digital/iiif/p16002coll4/2/14400,15150,1500,1500/300,/0/native.jpg


https://cdm16002.contentdm.oclc.org/digital/iiif/p16002coll4/2/3450,17325,1350,1350/300,/0/native.jpg


https://cdm16002.contentdm.oclc.org/digital/iiif/p16002coll4/2/12775,17350,1525,1525/300,/0/native.jpg


https://cdm16002.contentdm.oclc.org/digital/iiif/p16002coll4/3/15625,3000,1300,1300/300,/0/native.jpg


# Write Script to Save Image Files and Metadata

Was there something like this in one of the other notebooks?

### Use Case 1: A IIIF Memory Game

Write all face URLs to a Javascript file in order to feed them into a Memory game. This game actually uses IIIF Image API endpoints. 

In [31]:
!git clone https://github.com/leanderseige/cdvost2018_iiifworkshop.git iiifgame

Cloning into 'iiifgame'...
remote: Enumerating objects: 174, done.
remote: Total 174 (delta 0), reused 0 (delta 0), pack-reused 174
Receiving objects: 100% (174/174), 13.57 MiB | 30.20 MiB/s, done.
Resolving deltas: 100% (101/101), done.


In [32]:
cd iiifgame

/content/iiifgame


In [33]:
file = open("game/urls.js","w") 
file.write("var urls=[") 
for fid in faces_xy:
    if fid > 0:
        file.write(",")
    file.write('"'+faces_xy[fid]['quick_url']+'"')
file.write("];")  
file.close() 

HTML('<a target="_blank" href="game/game.html">Game</a>')

### Use Case 2: Random Avatar Generator.

Choose randomly the upper 50% of a face, the middle 20% of another face and the lower 30% of a third face and combine it to a random patchwork portrait. Click on the image parts to flip the image horizontally.

In [34]:
efile = open("generator/eurls.js","w") 
efile.write("var eurls=[") 

nfile = open("generator/nurls.js","w") 
nfile.write("var nurls=[") 

mfile = open("generator/murls.js","w") 
mfile.write("var murls=[") 

for fid in faces_xy:
    if fid > 0:
        efile.write(",")
        nfile.write(",")
        mfile.write(",")
    u = img_urls[faces_xy[fid]['pid']]
    x = faces_xy[fid]['x']
    y = faces_xy[fid]['y']
    w = faces_xy[fid]['w']
    h = faces_xy[fid]['h']
    efile.write('"'+u+"/%d,%d,%d,%d/300,/0/native.jpg"%(x,y,w,int(h*0.5))+'"');
    nfile.write('"'+u+"/%d,%d,%d,%d/300,/0/native.jpg"%(x,y+int(h*0.5),w,int(h*0.2))+'"');
    mfile.write('"'+u+"/%d,%d,%d,%d/300,/0/native.jpg"%(x,y+int(h*0.7),w,int(h*0.3))+'"');

efile.write("];")  
efile.close() 

nfile.write("];")  
nfile.close() 

mfile.write("];")  
mfile.close() 

HTML('<a target="_blank" href="generator/index.html">Generator</a>')